In [1]:
import pandas as pd
import pyodbc
import getpass

In [33]:
# Create connection

server = 'localhost'
database = 'test'
username = 'sa'
password = getpass.getpass()
cnxn = pyodbc.connect('DRIVER=/usr/local/lib/libmsodbcsql.17.dylib;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
#cursor.execute("SELECT @@version;")
#row = cursor.fetchone()
#while row:
#    print(row)
#    row = cursor.fetchone()


········


For a given CSV create the 'Create Table' script. 

In [34]:
filename = '/Users/bill/code/dataquest/data/food_info.csv' #in config
tablename = 'food_info' # string method needed
df = pd.read_csv(filename)
rows = df.shape[0] # to be inserted to a metadata col
cols = df.shape[1]
fields = df.columns

print(cols)
#print(df.columns)

36


In [36]:
sql = 'DROP TABLE IF EXISTS ' + tablename
cursor.execute(sql)
cursor.commit()

In [37]:
sql = ''
sqlend = ''
for counter, field in enumerate(fields):
    last_col = len(fields) - 1
    if counter < last_col:
        sql = sql + ('['+field+']' + ' varchar(500),') + '\n'
    else:
        sql = sql + ('['+field+']' + ' varchar(500)')

sql_create = 'CREATE TABLE ' + tablename + '(' + '\n' + sql + ')'

cursor.execute(sql_create)
cursor.commit()


In [48]:
bulk_insert = 'BULK INSERT ' + tablename + ' FROM '"'/food_info.csv'"' WITH (FIELDTERMINATOR = '"','"',ROWTERMINATOR = '"'0x0a'"');'

cursor.execute(bulk_insert)
cursor.commit()


In [49]:
cnxn.close()